In [1]:
import torch
import torch.nn as nn   
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import time
import random
import pandas as pd
import torch.nn.functional as F
device=torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
from tqdm import tqdm
import warnings 
warnings.filterwarnings("ignore")
cpac_root='/media/D/yazid/ASD-classification-ANEGCN/ABIDEI_CPAC/'
smri_root='/media/D/yazid/ASD-classification-ANEGCN/ABIDEI_sMRI/'
nan_subid=np.load('nan_subid.npy').tolist()
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

In [2]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention,self).__init__()
        self.conv1=nn.Conv1d(in_channels=3,out_channels=3,kernel_size=1,padding=0)
        self.conv2=nn.Conv1d(in_channels=116,out_channels=116,kernel_size=1,padding=0)
        self.softmax=nn.Softmax(dim=-1)
    def forward(self,Z,X):
        batchsize,x_dim,x_c= X.size()
        batchsize,z_dim,z_c= Z.size()
        K=self.conv1(X.permute(0,2,1))# BS,x_c,x_dim
        Q=K.permute(0,2,1)# BS,x_dim,x_c
        V=self.conv2(Z.permute(0,2,1))# Bs,z_c,z_dim
        attention=self.softmax(torch.matmul(Q,K))#BS,x_dim,x_dim
        out=torch.bmm(attention,V).permute(0,2,1)#BS,z_dim,z_c
        return out
class ANEGCN(nn.Module):
    def __init__(self,layer):
        super(ANEGCN,self).__init__()
        self.layer =layer
        self.relu  =nn.ReLU()
        self.atten =nn.ModuleList([Attention() for i in range(layer)])
        self.norm_n=nn.ModuleList([nn.BatchNorm1d(116) for i in range(layer)])
        self.norm_e=nn.ModuleList([nn.BatchNorm1d(116) for i in range(layer)])
        self.node_w=nn.ParameterList([nn.Parameter(torch.randn((3,3),dtype=torch.float32)) for i in range(layer)])
        self.edge_w=nn.ParameterList([nn.Parameter(torch.randn((116,116),dtype=torch.float32)) for i in range(layer)])
        self.line_n=nn.ModuleList([nn.Sequential(nn.Linear(116*3,128),nn.ReLU(),nn.BatchNorm1d(128)) for i in range(layer+1)])
        self.line_e=nn.ModuleList([nn.Sequential(nn.Linear(116*116,128*3),nn.ReLU(),nn.BatchNorm1d(128*3)) for i in range(layer+1)])
        self.clase =nn.Sequential(nn.Linear(128*4*(self.layer+1),1024),nn.Dropout(0.2),nn.ReLU(),
                                   nn.Linear(1024,2))
        self.ones=nn.Parameter(torch.ones((116),dtype=torch.float32),requires_grad=False)
        self._initialize_weights()
    # params initialization
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, (nn.Conv1d,nn.Linear)):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    def normalized(self,Z):
        n=Z.size()[0]
        A=Z[0,:,:]
        A=A+torch.diag(self.ones)
        d=A.sum(1)
        D=torch.diag(torch.pow(d,-1))
        A=D.mm(A).reshape(1,116,116)
        for i in range(1,n):
            A1=Z[i,:,:]+torch.diag(self.ones)
            d=A1.sum(1)
            D=torch.diag(torch.pow(d,-1))
            A1=D.mm(A1).reshape(1,116,116)
            A=torch.cat((A,A1),0)
        return A
        
    def update_A(self,Z):
        n=Z.size()[0]
        A=Z[0,:,:]
        Value,_=torch.topk(torch.abs(A.view(-1)),int(116*116*0.2))
        A=(torch.abs(A)>=Value[-1])+torch.tensor(0,dtype=torch.float32)
        A=A.reshape(1,116,116)
        for i in range(1,n):
            A2=Z[i,:,:]
            Value,_=torch.topk(torch.abs(A2.view(-1)),int(116*116*0.2))
            A2=(torch.abs(A2)>=Value[-1])+torch.tensor(0,dtype=torch.float32)
            A2=A2.reshape(1,116,116)
            A=torch.cat((A,A2),0)
        return A
        
    def forward(self,X,Z):
        n=X.size()[0]
        XX=self.line_n[0](X.view(n,-1))
        ZZ=self.line_e[0](Z.view(n,-1))
        for i in range(self.layer):
            A=self.atten[i](Z,X)
            Z1=torch.matmul(A,Z)
            Z2=torch.matmul(Z1,self.edge_w[i])
            Z=self.relu(self.norm_e[i](Z2))+Z
            ZZ=torch.cat((ZZ,self.line_e[i+1](Z.view(n,-1))),dim=1)
            X1=torch.matmul(A,X)
            X1=torch.matmul(X1,self.node_w[i])
            X=self.relu(self.norm_n[i](X1))+X
            #X.register_hook(grad_X_hook)
            #feat_X_hook(X)
            XX=torch.cat((XX,self.line_n[i+1](X.view(n,-1))),dim=1)
        XZ=torch.cat((XX,ZZ),1)
        y=self.clase(XZ)
        #print(self.clase[0].weight)
        return y
def grad_X_hook(grad):
    X_grad.append(grad)
def feat_X_hook(X):
    X_feat.append(X.detach())
X_grad=list()
X_feat=list()
class LabelSmoothLoss(nn.Module):
    
    def __init__(self, smoothing=0.0):
        super(LabelSmoothLoss, self).__init__()
        self.smoothing = smoothing
    
    def forward(self, input, target):
        log_prob = F.log_softmax(input, dim=-1)
        weight = input.new_ones(input.size()) * \
            self.smoothing / (input.size(-1) - 1.)
        weight.scatter_(-1, target.unsqueeze(-1), (1. - self.smoothing))
        loss = (-weight * log_prob).sum(dim=-1).mean()
        return loss

In [3]:
def cal_evaluate(TP,TN,FP,FN):
    if TP>0:
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        F1 = 2 * r * p / (r + p)
    else:
        F1=0
    acc = (TP + TN) / (TP + TN + FP + FN)
    #print('ACC:%.4f  F1:%.4f  [TP:%d|TN:%d|FP:%d|FN:%d]'%(acc,F1,TP,TN,FP,FN))
    return acc,F1
def test(device,model,testloader):
    model.eval()
    TP_test,TN_test,FP_test,FN_test=0,0,0,0
    with torch.no_grad():
        for (X,Z,label,sub_id) in testloader:
            TP,TN,FN,FP=0,0,0,0
            n=X.size()[0]
            X=X.to(device)
            Z=Z.to(device)
            label=label.to(device)
            y=model(X,Z)
            _,predict=torch.max(y,1)
            TP+=((predict==1)&(label==1)).sum().item()
            TN+=((predict==0)&(label==0)).sum().item()
            FN+=((predict==0)&(label==1)).sum().item()
            FP+=((predict==1)&(label==0)).sum().item()
            TP_test+=TP
            TN_test+=TN
            FP_test+=FP
            FN_test+=FN
        acc,f1=cal_evaluate(TP_test,TN_test,FP_test,FN_test)
        global max_acc
        global modelname
        global savedModel
        if acc>=max_acc:
            max_acc=acc
            if saveModel:
                torch.save(model.state_dict(),modelname)
        return acc,f1,TP_test,TN_test,FP_test,FN_test

In [4]:
class dataset(Dataset):
    def __init__(self,fmri_root,smri_root,site,ASD,TDC):
        super(dataset,self).__init__()
        self.fmri=fmri_root
        self.smri=smri_root
        self.ASD=[j for i in ASD for j in i]
        self.TDC=[j for i in TDC for j in i]
        self.data=self.ASD+self.TDC
        random.shuffle(self.data)
        self.data_site={}
        for i in range(len(site)):
            data=ASD[i]+TDC[i]
            for j in data:
                if j not in self.data_site:
                    self.data_site[j]=site[i]                
    def __getitem__(self,index):
        data=self.data[index]
        sub_id=int(data[0:5])
        if data in self.ASD:
            data_slow5 =np.load(self.fmri+self.data_site[data]+'/group1_slow5/'+data,allow_pickle=True)
            data_slow4 =np.load(self.fmri+self.data_site[data]+'/group1_slow4/'+data,allow_pickle=True)
            data_voxel =np.load(self.smri+self.data_site[data]+'/group1/'+data,allow_pickle=True)
            data_FCz   =np.load(self.fmri+self.data_site[data]+'/group1_FC/'+data,allow_pickle=True)
        elif data in self.TDC:
            data_slow5 =np.load(self.fmri+self.data_site[data]+'/group2_slow5/'+data,allow_pickle=True)
            data_slow4 =np.load(self.fmri+self.data_site[data]+'/group2_slow4/'+data,allow_pickle=True)
            data_voxel =np.load(self.smri+self.data_site[data]+'/group2/'+data,allow_pickle=True)
            data_FCz   =np.load(self.fmri+self.data_site[data]+'/group2_FC/'+data,allow_pickle=True)
        else:
            print('wrong input')
        data_slow5=(data_slow5-np.min(data_slow5))/(np.max(data_slow5)-np.min(data_slow5))
        data_slow4=(data_slow4-np.min(data_slow4))/(np.max(data_slow4)-np.min(data_slow4))
        if np.any(np.isnan(data_slow5)) or np.any(np.isnan(data_slow4)) or np.any(np.isnan(data_FCz)):
            print('data wronmg')
        #data_FCz=(data_FCz-np.min(data_FCz))/(np.max(data_FCz)-np.min(data_FCz))
        if self.data[index] in self.ASD:
            label=torch.tensor(1)
        else:
            label=torch.tensor(0)
        X=np.zeros((116,3),dtype=np.float32)
        X[:,0]=data_slow5
        X[:,1]=data_slow4
        X[:,2]=data_voxel
        data_FCz=data_FCz.astype(np.float32)
        Z=torch.from_numpy(data_FCz)
        X=torch.from_numpy(X)
        return X,Z,label,sub_id
    def __len__(self):
        return len(self.data)

In [5]:
def train_fgsm(model,trainloader,testloader,epsilon=0.05):
    result=pd.DataFrame(columns=('Loss','Acc','F1','TP','TN','FP','FN'))
    criterian1=LabelSmoothLoss(0.1).to(device)
    optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
    scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=gmma)
    acc=0.5000
    loss_sum=0
    for j in range(epoch):
        print('\rLoss: {:.2f}  Acc:{:.4f}'.format(loss_sum,acc),end='')
        loss_sum=0
        TP,TN,FP,FN=0,0,0,0
        model.train()
        for (X,Z,label,sub_id) in trainloader:
            x=X.to(device)
            z=Z.to(device)
            x.requires_grad=True
            z.requires_grad=True
            label=label.to(device)
            y=model(x,z)
            loss=criterian1(y,label)
            model.zero_grad()
            loss.backward(retain_graph=True)
            sign_grad_x=torch.sign(x.grad.data)
            sign_grad_z=torch.sign(z.grad.data)
            perturbed_x=x+epsilon*sign_grad_x 
            perturbed_z=z+epsilon*sign_grad_z 
            perturbed_x=torch.clamp(perturbed_x,0,1)
            perturbed_z=torch.clamp(perturbed_z,-1,1)
            y=model(perturbed_x,perturbed_z)
            L2=torch.tensor(0,dtype=torch.float32).to(device)
            if L2_lamda>0:
                for name,parameters in model.named_parameters():
                    if name[0:5]=='clase' and  name[-8:]=='0.weight':
                        L2+=L2_lamda*torch.norm(parameters,2)
            loss=0.5*(criterian1(y,label)+loss)+L2
            loss_sum+=loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if (j+1)%10 == 0 or j==0:
            acc,f1,TP_test,TN_test,FP_test,FN_test=test(device,model,testloader)
        result.loc[j]={'Loss':loss_sum,'Acc':acc,'F1':f1,'TP':TP_test,'TN':TN_test,'FP':FP_test,'FN':FN_test}
    result.sort_values('Acc',inplace=True,ascending=False)
    print('',end='')
    print('Acc:  ', result.iloc[0]['Acc'])
def train_norm(model,trainloader,testloader):
    result=pd.DataFrame(columns=('Loss','Acc','F1','TP','TN','FP','FN'))
    criterian1=LabelSmoothLoss(0.1).to(device)
    optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
    acc=0.5000
    loss_sum=0
    for j in range(epoch):
        print('\rEPOCH: [{:03d}|100]  Loss: {:.2f}  Acc:{:.4f}'.format(j+1,loss_sum,acc),end='')
        loss_sum=0
        TP,TN,FP,FN=0,0,0,0
        model.train()
        for (X,Z,label,sub_id) in trainloader:
            x=X.to(device)
            z=Z.to(device)
            label=label.to(device)
            y=model(x,z)
            loss=criterian1(y,label)
            L2=torch.tensor(0,dtype=torch.float32).to(device)
            if L2_lamda>0:
                for name,parameters in model.named_parameters():
                    if name[0:5]=='clase' and  name[-8:]=='0.weight':
                        L2+=L2_lamda*torch.norm(parameters,2)
            loss=loss+L2
            loss_sum+=loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if (j+1)%10 == 0 or j==0:
            acc,f1,TP_test,TN_test,FP_test,FN_test=test(device,model,testloader)
            result.loc[j//10]={'Loss':loss_sum,'Acc':acc,'F1':f1,'TP':TP_test,'TN':TN_test,'FP':FP_test,'FN':FN_test}
    result.sort_values('Acc',inplace=True,ascending=False)
    print(' FinalAcc: {:.4f}'.format(result.iloc[0]['Acc']))
def train_pgd(model,trainloader,testloader,eps=0.2,iters=10,alpha=8/255):
    result=pd.DataFrame(columns=('Loss','Acc','F1','TP','TN','FP','FN'))
    criterian1=LabelSmoothLoss(0.1).to(device)
    optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
    for j in range(epoch):
        loss_sum=0.
        TP,TN,FP,FN=0,0,0,0
        model.train()
        for (X,Z,label,sub_id) in trainloader:
            model.train()
            x=X.to(device)
            z=Z.to(device)
            label=label.to(device)
            pretu_x,pretu_z=x,z
            ori_x,ori_z=x.data,z.data
            for i in range(iters):
                pretu_x.requires_grad=True
                pretu_z.requires_grad=True
                y=model(pretu_x,pretu_z)
                loss=criterian1(y,label)
                model.zero_grad()
                loss.backward()
                adv_x=pretu_x+alpha*torch.sign(pretu_x.grad.data)
                adv_z=pretu_z+alpha*torch.sign(pretu_z.grad.data)
                eta_x=torch.clamp(adv_x-ori_x,min=-eps,max=eps)
                eta_z=torch.clamp(adv_z-ori_z,min=-eps,max=eps)
                pretu_x=torch.clamp(ori_x+eta_x,min=0,max=1).detach_()
                pretu_z=torch.clamp(ori_z+eta_z,min=-1,max=1).detach_()
            y=model(x,z)
            yy=model(pretu_x,pretu_z)
            L2=torch.tensor(0,dtype=torch.float32).to(device)
            if L2_lamda>0:
                for name,parameters in model.named_parameters():
                    if name[0:5]=='clase' and  name[-8:]=='0.weight':
                        L2+=L2_lamda*torch.norm(parameters,2)
            loss=0.5*(criterian1(yy,label)+criterian1(y,label))+L2
            loss_sum+=loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if (j+1)%10==0:
            acc,f1,TP_test,TN_test,FP_test,FN_test=test(device,model,testloader)
            result.loc[(j+1)//10]=[loss_sum,acc,f1,TP_test,TN_test,FP_test,FN_test]
    result.sort_values('Acc',inplace=True,ascending=False)
    print(' FinalAcc: {:.4f}'.format(result.iloc[0]['Acc']))
    return result.iloc[0]['Acc']

In [6]:
## 机构混合续
train_site=test_site=np.load('DATAARRANGE/train_test_site.npy')
train_asd_dict=np.load('DATAARRANGE/train_asd_dict.npy',allow_pickle=True).item()
train_tdc_dict=np.load('DATAARRANGE/train_tdc_dict.npy',allow_pickle=True).item()
test_asd_dict=np.load('DATAARRANGE/test_asd_dict.npy',allow_pickle=True).item()
test_tdc_dict=np.load('DATAARRANGE/test_tdc_dict.npy',allow_pickle=True).item()

In [7]:
setup_seed(123)
global max_acc
global saveModel
global modelname
for i in range(13):
    L1_lamda=0.0
    L2_lamda=0.0001
    learning_rate=0.0001
    epoch   =100
    batch_size=64
    layer   =5
    for index in range(10):
        saveModel=True
        max_acc=0.6
        modelname='../SAVEDModels/PGDtrainedensamble/models_{}_{}'.format(i,index)
        train_asd=train_asd_dict[index]
        train_tdc=train_tdc_dict[index]
        test_asd =test_asd_dict[index]
        test_tdc =test_tdc_dict[index]
        trainset=dataset(site=train_site,fmri_root=cpac_root,smri_root=smri_root,ASD=train_asd,TDC=train_tdc)
        trainloader=DataLoader(trainset,batch_size=batch_size,shuffle=True)
        testset=dataset(site=test_site,fmri_root=cpac_root,smri_root=smri_root,ASD=test_asd,TDC=test_tdc)
        testloader=DataLoader(testset,batch_size=1)
        model=ANEGCN(layer).to(device)
        train_pgd(model,trainloader,testloader,eps=0.02,iters=10,alpha=0.004)

 FinalAcc: 0.7765
 FinalAcc: 0.6863
 FinalAcc: 0.6634
 FinalAcc: 0.6893
 FinalAcc: 0.7670
 FinalAcc: 0.6875
 FinalAcc: 0.7200
 FinalAcc: 0.6346
 FinalAcc: 0.6869
 FinalAcc: 0.6842
 FinalAcc: 0.7529
 FinalAcc: 0.7157
 FinalAcc: 0.7030
 FinalAcc: 0.7087
 FinalAcc: 0.7282
 FinalAcc: 0.7083
 FinalAcc: 0.6900
 FinalAcc: 0.6538
 FinalAcc: 0.6768
 FinalAcc: 0.6930
 FinalAcc: 0.7412
 FinalAcc: 0.7255
 FinalAcc: 0.7426
 FinalAcc: 0.6990
 FinalAcc: 0.6990
 FinalAcc: 0.7083
 FinalAcc: 0.7000
 FinalAcc: 0.6731
 FinalAcc: 0.6970
 FinalAcc: 0.6579
 FinalAcc: 0.7529
 FinalAcc: 0.6863
 FinalAcc: 0.6931
 FinalAcc: 0.6893
 FinalAcc: 0.7087
 FinalAcc: 0.6979
 FinalAcc: 0.6900
 FinalAcc: 0.6250
 FinalAcc: 0.6768
 FinalAcc: 0.7018
 FinalAcc: 0.7647
 FinalAcc: 0.7353
 FinalAcc: 0.7228
 FinalAcc: 0.6990
 FinalAcc: 0.7184
 FinalAcc: 0.7083
 FinalAcc: 0.7100
 FinalAcc: 0.6635
 FinalAcc: 0.7374
 FinalAcc: 0.7193
 FinalAcc: 0.7765
 FinalAcc: 0.7647
 FinalAcc: 0.6832
 FinalAcc: 0.7184
 FinalAcc: 0.6990
 FinalAcc:

In [8]:
Pred_label={}
True_label={}
vote=13
for i in range(vote):
    Pred_label={}
    True_label={}
    for index in range(10):
        PATH='../SAVEDModels/PGDtrainedensamble/models_{}_{}'.format(i,index)
        test_asd =test_asd_dict[index]
        test_tdc =test_tdc_dict[index]
        testset=dataset(site=test_site,fmri_root=cpac_root,smri_root=smri_root,ASD=test_asd,TDC=test_tdc)
        testloader=DataLoader(testset,batch_size=1)
        model=ANEGCN(5)
        model.load_state_dict(torch.load(PATH))
        model.eval()
        with torch.no_grad():
            for (X,Z,label,sub_id) in testloader:
                True_label[sub_id.item()]=label.item()
                y=model(X,Z)
                _,predict=torch.max(y,1)
                if sub_id.item() not in Pred_label:
                    Pred_label[sub_id.item()]=0
                if predict.item()==1:
                    Pred_label[sub_id.item()]+=1      
TP,TN,FP,FN=0,0,0,0
for sId in True_label:
    if True_label[sId]==1:
        if Pred_label[sId]>= ((vote+1)//2):
            TP+=1
        else:
            FN+=1
    else:
        if Pred_label[sId]<= ((vote-1)//2):
            TN+=1
        else:
            FP+=1
print((TP+TN)/1007)
print(TP/(TP+FN))
print(TN/(FP+TN))

0.7358490566037735
0.6798336798336798
0.7870722433460076


03: 0.7229;
07: 0.7319;0.6736;0.7852;
13: 0.7329;0.6881;0.7738;
15: 0.7309;0.6861;0.7717;
17: 0.7269;0.6798;0.7700;
19: 0.7239;0.6775;0.7662;
21: 0.7269;0.6819;0.7681;
23: 0.7239;0.6840;0.7605;

13: 0.7468 0.7173 0.7738;

In [10]:
vote=13
for i in range(vote):
    Acc,Sen,Spe=np.zeros(10),np.zeros(10),np.zeros(10)
    for index in range(10):
        PATH='../SAVEDModels/PGDtrainedensamble/models_{}_{}'.format(i,index)
        test_asd =test_asd_dict[index]
        test_tdc =test_tdc_dict[index]
        testset=dataset(site=test_site,fmri_root=cpac_root,smri_root=smri_root,ASD=test_asd,TDC=test_tdc)
        testloader=DataLoader(testset,batch_size=1)
        model=ANEGCN(5)
        model.load_state_dict(torch.load(PATH))
        model.eval()
        model=model.to(device)
        with torch.no_grad():
            acc,f1,TP_test,TN_test,FP_test,FN_test=test(device,model,testloader)
            Acc[index]=acc
            Sen[index]=TP_test/(TP_test+FN_test)
            Spe[index]=TN_test/(TN_test+FP_test)
    print('Acc: %.4f+%.4f  Sen: %.4f+%.4f  Spe: %.4f+%.4f'%(Acc.mean(),Acc.std(),Sen.mean(),Sen.std(),Spe.mean(),Spe.std()))

Acc: 0.6996+0.0415  Sen: 0.6698+0.0771  Spe: 0.7268+0.0702
Acc: 0.7030+0.0259  Sen: 0.6305+0.0560  Spe: 0.7688+0.0477
Acc: 0.7044+0.0256  Sen: 0.6602+0.0970  Spe: 0.7442+0.0728
Acc: 0.6922+0.0298  Sen: 0.6183+0.1002  Spe: 0.7579+0.0887
Acc: 0.7179+0.0252  Sen: 0.6572+0.1010  Spe: 0.7750+0.0571
Acc: 0.7069+0.0379  Sen: 0.6794+0.0766  Spe: 0.7311+0.0842
Acc: 0.7072+0.0420  Sen: 0.6651+0.1061  Spe: 0.7464+0.0674
Acc: 0.7011+0.0326  Sen: 0.6562+0.0630  Spe: 0.7409+0.0696
Acc: 0.7094+0.0306  Sen: 0.6657+0.1211  Spe: 0.7506+0.0757
Acc: 0.7107+0.0355  Sen: 0.6757+0.0976  Spe: 0.7414+0.1029
Acc: 0.7099+0.0242  Sen: 0.6797+0.0601  Spe: 0.7366+0.0465
Acc: 0.7152+0.0501  Sen: 0.6783+0.0989  Spe: 0.7472+0.0879
Acc: 0.7445+0.0929  Sen: 0.6958+0.1601  Spe: 0.7891+0.0919


In [2]:
import time 
import numpy as np
A=np.random.randn(100000)
start=time.time()
B=2*A
end=time.time()
time1=end-start
start=time.time()
B=[2*i for i in A]
end=time.time()
time2=end-start
print(time1,time2)

0.0011470317840576172 0.06524252891540527
